# KFold + Catboost Baseline + Advanced Feature Engineering
Накручиваем поверх Baseline продвинутое FE.

In [1]:
from IPython.lib.deepreload import reload
%load_ext autoreload
%autoreload 2

import joblib
import numpy as np
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import warnings

from catboost import CatBoostRegressor
from sklearn import set_config
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder

# Вместо одного фиксированного разбиения на train/test используем стабильную стратегию кросс-валидации.
# Используем тут Cross-validation, потому что:
# 	•	нужно надёжно сравнить несколько разных моделей или гиперпараметров и понять, какая модель стабильнее и лучше в целом.
# 	•	хотим избежать случайных удач или провалов, связанных с конкретным разбиением на train/test.
# 	•	выбираем модель или гиперпараметры, которые потом будешь использовать для финального сабмишна на Kaggle.
# Делаем эту оценку, чтобы в дальнейших блокнотах-улучшениях сравнивать более корректно.
from sklearn.model_selection import KFold, RepeatedKFold, cross_val_score, train_test_split

# Используем IterativeImputer:
# 	•	Он итеративно заполняет все пропуски сразу.
# 	•	Работает одновременно со всеми признаками, учитывая связи между ними.
# 	•	Не требует ручного управления порядком заполнения.
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from utils.data_manager import DataManager, CrossFoldEncoder
from utils.model_manager import ModelManager
from utils.syth_generator_gaussian import CombinedSyntheticGenerator


# Зависимости для FE
from pandas.api.types import CategoricalDtype

from category_encoders import MEstimateEncoder
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [2]:
# --- Глобально включаем вывод Pandas для всех трансформеров ---
# (Можно применять и к отдельным трансформерам/пайплайнам .set_output(transform="pandas"))
set_config(transform_output = "pandas")

In [3]:
dm = DataManager()
mm = ModelManager()

# Отключаем автологгирование, чтобы использовать ручное
mlflow.sklearn.autolog(disable=True)
warnings.filterwarnings("ignore", module="mlflow")  # Игнорируем предупреждения MLflow


In [4]:
RANDOM_STATE = 42
N_FOLDS = 5  # Например, 5 или 10

In [5]:
# # Корреляционная матрица дата-сета
# dm.corrplot(train_data, annot=None)

## Предобработка данных 

In [6]:
# A label encoding is okay for any kind of categorical feature when you're using a tree-ensemble like XGBoost, even for unordered categories. If you wanted to try a linear regression model (also popular in this competition), you would instead want to use a one-hot encoding, especially for the features with unordered categories.
# X_prep = dm.label_encode(X_prep)

In [7]:
# # OHE по факту
# print(X_prep.BldgType.unique())
# tmp_df = pd.get_dummies(X_prep.BldgType, prefix="Bldg")
# tmp_df

In [105]:
# Базовые фичи, получаемы манипуляцией с данными
def mathematical_transforms(df):
    X = pd.DataFrame()  # dataframe to hold new features
    X["LivLotRatio"] = df.GrLivArea / df.LotArea
    X["Spaciousness"] = (df['1stFlrSF'] + df['2ndFlrSF']) / df.TotRmsAbvGrd
    return X

def interactions(df):
    # Получим новые признаки, которые сочетают в себе категориальную и числовую информацию. Модель сможет учитывать 
    # не просто тип здания, но и то, как конкретный тип здания влияет на значение площади и, соответственно, на целевую переменную.
    # Создаем бинарные признаки для каждого уникального типа здания:
    X = pd.get_dummies(df.BldgType, prefix="Bldg")  # Тип жилого здания
    # Каждый бинарный признак умножается на площадь здания
    X = X.mul(df.GrLivArea, axis=0)  # Жилая площадь над уровнем земли

    # Район в пределах города + общая классификация зонирования продажи
    X['Neighborhood_Zoning'] = df['Neighborhood'].astype(str) + '_' + df['MSZoning'].astype(str)

    # Тип продажи + Условия продажи
    X['SaleType_Condition'] = df['SaleType'].astype(str) + '_' + df['SaleCondition'].astype(str)

    quality_dict = {'Ex': 5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1, np.nan:0}  # Словарь качественных признаков
    X['TotalQualScore'] = 0
    # Качество внешней отделки; Качество кухни; Высота подвала; Качество и состояние отопления; Качество гаража;
    # Качество камина
    quality_cols = ['ExterQual', 'KitchenQual', 'BsmtQual', 'HeatingQC', 'GarageQual', 'FireplaceQu']
    for col in quality_cols:
        if col in df.columns:
             X['TotalQualScore'] += df[col].map(quality_dict).fillna(0)

    X['PorchDeckArea'] = 0
    # Площадь деревянной террасы; Площадь открытой террасы; Площадь закрытой террасы; Площадь 3-сезонной террасы; Площадь экрана
    porch_cols = ['WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch']
    for col in porch_cols:
         if col in df.columns:
            X['PorchDeckArea'] += df[col].fillna(0)
    X['HasPorchDeck'] = (X['PorchDeckArea'] > 0).astype(int)

    # Количество каминов
    X['HasFireplace'] = (df['Fireplaces'] > 0).astype(int)

    # Тип гаража
    X['HasGarage'] = (~df['GarageType'].isna()).astype(int)

    # Качество забора
    X['HasFence'] = (~df['Fence'].isna()).astype(int)

    return X

def counts(df):
    X = pd.DataFrame()
    X["PorchTypes"] = df[[
        "WoodDeckSF",
        "OpenPorchSF",
        "EnclosedPorch",
        "3SsnPorch",
        "ScreenPorch",
    ]].gt(0.0).sum(axis=1)
    return X

def break_down(df):
    X = pd.DataFrame()
    X["MSClass"] = df.MSSubClass.str.split("_", n=1, expand=True)[0]
    return X

def group_transforms(df):
    X = pd.DataFrame()
    X["MedNhbdArea"] = df.groupby("Neighborhood")["GrLivArea"].transform("median")
    return X

In [9]:
# K-Means
cluster_features = [
    "LotArea",  # Площадь участка в квадратных футах
    "TotalBsmtSF",  # Общая площадь подвала
    "1stFlrSF",  # Площадь первого этажа
    "2ndFlrSF",  # Площадь второго этажа
    "GrLivArea",  # Жилая площадь над уровнем земли
]


def cluster_labels(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=n_clusters, n_init=50, random_state=0)
    X_new = pd.DataFrame()
    X_new["Cluster"] = kmeans.fit_predict(X_scaled)
    return X_new


def cluster_distance(df, features, n_clusters=20):
    X = df.copy()
    X_scaled = X.loc[:, features]
    X_scaled = (X_scaled - X_scaled.mean(axis=0)) / X_scaled.std(axis=0)
    kmeans = KMeans(n_clusters=20, n_init=50, random_state=0)
    X_cd = kmeans.fit_transform(X_scaled)
    # Label features and join to dataset
    X_cd = pd.DataFrame(
        X_cd, columns=[f"Centroid_{i}" for i in range(X_cd.shape[1])]
    )
    return X_cd

In [10]:
# PCA
def apply_pca(X, standardize=True):
    # Standardize
    if standardize:
        X = (X - X.mean(axis=0)) / X.std(axis=0)
    # Create principal components
    pca = PCA()
    X_pca = pca.fit_transform(X)
    # Convert to dataframe
    component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
    X_pca = pd.DataFrame(X_pca, columns=component_names)
    # Create loadings
    loadings = pd.DataFrame(
        pca.components_.T,  # transpose the matrix of loadings
        columns=component_names,  # so the columns are the principal components
        index=X.columns,  # and the rows are the original features
    )
    return pca, X_pca, loadings


def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

def pca_inspired(df):
    X = pd.DataFrame()
    X["Feature1"] = df.GrLivArea + df.TotalBsmtSF
    X["Feature2"] = df.YearRemodAdd * df.TotalBsmtSF
    return X


def pca_components(df, features):
    X = df.loc[:, features]
    _, X_pca, _ = apply_pca(X)
    return X_pca


pca_features = [
    "GarageArea",  # Площадь гаража
    "YearRemodAdd",  # Год последнего ремонта
    "TotalBsmtSF",  # Общая площадь подвала
    "GrLivArea",  # Жилая площадь над уровнем земли
]

In [11]:
def indicate_outliers(df):
    """Создает бинарные фичи для выбросов.
    
    У нас есть выбросы, например, дома, которые продаются из частичной собственности. Они имеют огромную площадь, но т.к. частичная продажа - стоят очень дешево. Это мы выяснили благодаря PCA: см. https://alex-podrabinovich.medium.com/%D0%BF%D1%80%D0%BE%D0%B4%D0%B2%D0%B8%D0%BD%D1%83%D1%82%D1%8B%D0%B9-feature-engineering-1e402a4d52a9 
    Удалять выбросы мы не будем, конечно, но промаркируем их, чтобы модель могла с ними работать.
    """
    X_new = pd.DataFrame()
    X_new["Outlier"] = (df.Neighborhood == "Edwards") & (df.SaleCondition == "Partial")
    return X_new

In [12]:
# Создаем preprocessor с разными трансформерами для разных типов данных
# Числовые данные пропущенные предсказываем с помощью модели RandomForestRegressor

# Пайплайн для числовых признаков (итеративное заполнение)
numeric_transformer = Pipeline(steps=[
    ('imputer', IterativeImputer(
        estimator=RandomForestRegressor(n_estimators=50, random_state=RANDOM_STATE),
        max_iter=10,
        random_state=RANDOM_STATE
    )),  # Дает примерно +100 прирост качества vs mean/median
    # ('imputer', SimpleImputer(strategy='mean')),
    # ('scaler', StandardScaler()) 
])

# Пайплайн для категориальных признаков (заполнение частым значением и кодирование)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), 
    # ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [13]:
def load_data():
    # Read
    data_path = 'data/home-data-for-ml-course'
    df_train = pd.read_csv(data_path + '/train.csv', index_col="Id")
    df_test = pd.read_csv(data_path + '/test.csv', index_col="Id")

    # Merge the splits so we can process them together
    df = pd.concat([df_train, df_test])
    
    # Preprocessing
    numeric_columns, non_numeric_columns = dm.get_numeric_and_categorical_features(df)
    preprocessor = ColumnTransformer(
        transformers=[
            # Применяем к исходным числовым колонкам
            ('num', numeric_transformer, numeric_columns),
            # Применяем к исходным категориальным колонкам
            ('cat', categorical_transformer, non_numeric_columns)
        ],
        remainder='drop',   # 'passthrough' сохранит полиномиальные и другие колонки, которые не были ни числовыми, ни категориальными ИЗНАЧАЛЬНО
        verbose_feature_names_out=False  # Чтобы имена колонок не менялись на 'num__colname' и т.д.
    )
    df = preprocessor.fit_transform(df)

    # Reform splits
    df_train = df.loc[df_train.index, :]
    df_test = df.loc[df_test.index, :]
    
    return df_train, df_test

In [14]:
df_train, df_test = load_data()

In [114]:
def create_features(df, df_test=None):
    X = df.copy()
    y = X.pop("SalePrice")
    
    # if df_test is not None:
    #     return X, df_test
    # return X
    
    print(f"Initial X shape: {X.shape}")
    
    mi_scores = dm.make_mi_scores(X, y)
        
    # Combine splits if test data is given
    #
    # If we're creating features for test set predictions, we should
    # use all the data we have available. After creating our features,
    # we'll recreate the splits.
    if df_test is not None:
        X_test = df_test.copy()
        X_test.pop("SalePrice")
        X = pd.concat([X, X_test])  
        print(f"Final X shape after combine with test: {X.shape}")

    # Mutual Information
    # print(f"Dropping useless features from train set: {X.shape}")
    # X = dm.drop_uninformative(X, mi_scores)
    # print(f"Without useless features: {X.shape}")

    # Transformations
    X = X.join(mathematical_transforms(X))
    X = X.join(interactions(X))
    X = X.join(counts(X))
    # X = X.join(break_down(X))
    X = X.join(group_transforms(X))
    print(f"X shape after Transformation: {X.shape}")

    # Clustering
    # X = X.join(cluster_labels(X, cluster_features, n_clusters=20))
    # X = X.join(cluster_distance(X, cluster_features, n_clusters=20))
    print(f"X shape after K-Means: {X.shape}")

    # PCA
    X = X.join(pca_inspired(X))
    # X = X.join(pca_components(X, pca_features))
    # X = X.join(indicate_outliers(X))
    print(f"X shape after PCA: {X.shape}")

    X, _ = dm.label_encode(X)
    print(f"X shape after Label Encoding: {X.shape}")

    # Reform splits
    if df_test is not None:
        X_test = X.loc[df_test.index, :]
        X.drop(df_test.index, inplace=True)
    print(f"X shape after Reform splits: {X.shape}")
        
    # # Target Encoder
    # encoder = CrossFoldEncoder(MEstimateEncoder, m=1)
    # X = X.join(encoder.fit_transform(X, y, cols=["MSSubClass"]))  # Тип жилья, участвующего в продаже
    # if df_test is not None:
    #     X_test = X_test.join(encoder.transform(X_test))
    # print(f"X shape after Target encoding: {X.shape}")

    if df_test is not None:
        return X, X_test
    else:
        return X

In [115]:
X_train = create_features(df_train)
y_train = df_train.loc[:, "SalePrice"]
print(X_train.shape)
X_train.head()

Initial X shape: (1460, 79)
X shape after Transformation: (1460, 96)
X shape after K-Means: (1460, 96)
X shape after PCA: (1460, 98)
X shape after Label Encoding: (1460, 98)
X shape after Reform splits: (1460, 98)
(1460, 98)


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,TotalQualScore,PorchDeckArea,HasPorchDeck,HasFireplace,HasGarage,HasFence,PorchTypes,MedNhbdArea,Feature1,Feature2
Id,,,,,,,,,,,,,,,,,,,,,
1,60.0,65.0,8450.0,7.0,5.0,2003.0,2003.0,196.0,706.0,0.0,...,24,61.0,1,0,1,1,1,1500.0,2566.0,1714568.0
2,20.0,80.0,9600.0,6.0,8.0,1976.0,1976.0,0.0,978.0,0.0,...,21,298.0,1,1,1,1,1,1437.0,2524.0,2493712.0
3,60.0,68.0,11250.0,7.0,5.0,2001.0,2002.0,162.0,486.0,0.0,...,23,42.0,1,1,1,1,1,1500.0,2706.0,1841840.0
4,70.0,60.0,9550.0,7.0,5.0,1915.0,1970.0,0.0,216.0,0.0,...,21,307.0,1,1,1,1,2,1717.0,2473.0,1489320.0
5,60.0,84.0,14260.0,8.0,5.0,2000.0,2000.0,350.0,655.0,0.0,...,23,276.0,1,1,1,1,2,2418.0,3343.0,2290000.0


### Валидация

In [108]:
catboost_params = {
    'iterations': 1000, 
    'learning_rate': 0.05, 
    'depth': 6, 
    'loss_function': 'RMSE', 
    'verbose': 0, 
    'random_seed': RANDOM_STATE,
    'early_stopping_rounds': 100  # ?
}

In [109]:
"""
Пояснения к тому, что тут происходит:

Тут мы ОЦЕНИВАЕМ и ТЕСТИРУЕМ подобранные фичи и гиперпараметры. В цикле бежим по N_FOLD-ам, бьем данные тренировочные в соотношении 80% на обучение, 20% на валидацию. Обучаем модель на трейне, валидируем на валидации. Смотрим на ошибки. Если по итогу нас все устраивает, мы дальше идем и обучаем модель с нуля на всех данных со всеми гиперпараметрами и фичами (да еще можно вычленить из цикла итерацию самую лучшую для нужного количества итераций обучения финальной модели),

oof_predictions — это массив, который в итоге будет содержать предсказания для каждого объекта из исходного тренировочного набора (X). Важно, что предсказание для конкретного объекта (например, дома №100) делается той моделью (из цикла CV), которая обучалась без этого объекта. Надежная оценка качества: OOF-предикты позволяют посчитать метрику качества (например, oof_rmse) на всем тренировочном наборе, при этом каждое предсказание было сделано "честно" (модель не видела этот объект при обучении). 
Эта оценка часто бывает более надежной, чем простое усреднение метрик по фолдам (mean_cv_rmse), так как она считается на полном наборе данных один раз. 
Можно сравнить oof_predictions с реальными значениями y_log (или y), чтобы понять, на каких объектах модель ошибается сильнее всего.
Стэкинг/Блендинг: OOF-предикты часто используются как новые признаки для обучения модели второго уровня (мета-модели) в ансамблях (стэкинг).
"""

# --- Кросс-Валидация ---
kf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDOM_STATE)

oof_predictions = np.zeros(X_train.shape[0])  # Для хранения out-of-fold предсказаний
fold_rmses = []
fold_best_iterations = []  # Будем сохранять лучшие итерации

mlflow.set_experiment("KFold Default With improved features v1")
with mlflow.start_run() as run:  # Сохраняем run для логирования артефактов
    for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train)): 
        print(f"--- Fold {fold+1}/{N_FOLDS} ---")
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Обучаем модель с early stopping
        model = CatBoostRegressor(**catboost_params)
        model.fit(X_train_fold, y_train_fold,
                  eval_set=[(X_val_fold, y_val_fold)],
                  verbose=0,  # Отключаем вывод, 100 - если хотим видеть обучение 
                 )

        # Сохраняем лучшую итерацию
        best_iter = model.get_best_iteration()
        fold_best_iterations.append(best_iter)
        print(f"Best iteration for fold {fold+1}: {best_iter}")

        # 4. Предсказания на валидации
        val_preds = model.predict(X_val_fold)
        oof_predictions[val_idx] = val_preds

        # Оценка на фолде
        fold_rmsle = np.sqrt(mean_squared_error(np.log(y_val_fold), np.log(val_preds)))
        print(f"Fold {fold+1} RMSLE: {fold_rmsle}")
        fold_rmses.append(fold_rmsle)
        mlflow.log_metric(f"fold_{fold+1}_rmsle", fold_rmsle, step=fold+1)
        mlflow.log_metric(f"fold_{fold+1}_best_iter", best_iter, step=fold+1)
    
    # --- Итоговая оценка CV ---
    mean_cv_rmse = np.mean(fold_rmses)
    std_cv_rmse = np.std(fold_rmses)
    oof_rmse = np.sqrt(mean_squared_error(np.log(y_train), np.log(oof_predictions)))

    print(f"\nMean CV RMSLE: {mean_cv_rmse:.4f} +/- {std_cv_rmse:.4f}")
    print(f"OOF RMSLE: {oof_rmse:.4f}")
    print(f"Mean best iteration: {np.mean(fold_best_iterations):.0f}")

    # Логгирование итоговых метрик вручную
    mlflow.log_metric("mean_cv_rmse", mean_cv_rmse)
    mlflow.log_metric("std_cv_rmse", std_cv_rmse)
    mlflow.log_metric("oof_rmse", oof_rmse)
    mlflow.log_metric("mean_best_iteration", np.mean(fold_best_iterations))

--- Fold 1/5 ---
Best iteration for fold 1: 751
Fold 1 RMSLE: 0.13268481962928946
--- Fold 2/5 ---
Best iteration for fold 2: 821
Fold 2 RMSLE: 0.10707061671863766
--- Fold 3/5 ---
Best iteration for fold 3: 62
Fold 3 RMSLE: 0.16698609472454448
--- Fold 4/5 ---
Best iteration for fold 4: 903
Fold 4 RMSLE: 0.12427656760319122
--- Fold 5/5 ---
Best iteration for fold 5: 984
Fold 5 RMSLE: 0.10361146417006144

Mean CV RMSLE: 0.1269 +/- 0.0227
OOF RMSLE: 0.1289
Mean best iteration: 704


In [102]:
# К чему стремимся
# --- Fold 1/5 ---
# Best iteration for fold 1: 626
# Fold 1 RMSLE: 0.12822565845390327
# --- Fold 2/5 ---
# Best iteration for fold 2: 916
# Fold 2 RMSLE: 0.10662773995346779
# --- Fold 3/5 ---
# Best iteration for fold 3: 594
# Fold 3 RMSLE: 0.13932657522708686
# --- Fold 4/5 ---
# Best iteration for fold 4: 710
# Fold 4 RMSLE: 0.12416343423096966
# --- Fold 5/5 ---
# Best iteration for fold 5: 997
# Fold 5 RMSLE: 0.10615331382371691
# 
# Mean CV RMSLE: 0.1209 +/- 0.0128
# OOF RMSLE: 0.1216
# Mean best iteration: 769



In [67]:
# Hyperparameter Tuning here...
# X_train = create_features(df_train)
# y_train = df_train.loc[:, "SalePrice"]
# 
# xgb_params = dict(
#     max_depth=6,           # maximum depth of each tree - try 2 to 10
#     learning_rate=0.01,    # effect of each tree - try 0.0001 to 0.1
#     n_estimators=1000,     # number of trees (that is, boosting rounds) - try 1000 to 8000
#     min_child_weight=1,    # minimum number of houses in a leaf - try 1 to 10
#     colsample_bytree=0.7,  # fraction of features (columns) per tree - try 0.2 to 1.0
#     subsample=0.7,         # fraction of instances (rows) per tree - try 0.2 to 1.0
#     reg_alpha=0.5,         # L1 regularization (like LASSO) - try 0.0 to 10.0
#     reg_lambda=1.0,        # L2 regularization (like Ridge) - try 0.0 to 10.0
#     num_parallel_tree=1,   # set > 1 for boosted random forests
# )
# 
# xgb = XGBRegressor(**xgb_params)
# score_dataset(X_train, y_train, xgb)

### Финальная модель и submission

In [110]:
# Train Model and Create Submissions
X_train, X_test = create_features(df_train, df_test)
y_train = df_train.loc[:, "SalePrice"]

final_iterations = 1000
final_catboost_params = catboost_params.copy()
final_catboost_params['iterations'] = final_iterations
final_catboost_params.pop('early_stopping_rounds', None)  # Убираем early stopping для финального обучения
print(f"Финальные параметры обучения модели: {final_catboost_params}")

final_model = CatBoostRegressor(**final_catboost_params)

final_model.fit(X_train, y_train)
print("Final model trained.")

Initial X shape: (1460, 79)
Final X shape after combine with test: (2919, 79)
X shape after Transformation: (2919, 96)
X shape after K-Means: (2919, 96)
X shape after PCA: (2919, 98)
X shape after Label Encoding: (2919, 98)
X shape after Reform splits: (1460, 98)
X shape after Target encoding: (1460, 99)
Финальные параметры обучения модели: {'iterations': 1000, 'learning_rate': 0.05, 'depth': 6, 'loss_function': 'RMSE', 'verbose': 0, 'random_seed': 42}
Final model trained.


In [111]:
# Предсказание на тест данных для сабмишна
predictions = final_model.predict(X_test)
submission_single = pd.DataFrame({'Id': X_test.index, 'SalePrice': predictions})
submission_single.to_csv('final_FE_v13.csv', index=False)
submission_single.head()

,Id,SalePrice
0,1461,120825.414040
1,1462,166370.184723
2,1463,190696.698673
3,1464,198871.919007
4,1465,187180.335068


In [ ]:
# ==============================================================
# ==============================================================
# ==============================================================

In [17]:
# Kaggle BASELINE

# 12879.10346